# Uber Movement

This notebook prepares the data pulled from Uber Movement. This data can be found [here](https://movement.uber.com/explore/washington_DC/travel-times/query?si=186&ti=&ag=censustracts&dt[tpb]=ALL_DAY&dt[dr][sd]=2020-03-01&dt[dr][ed]=2020-03-31&dt[wd;]=1,2,3,4,5,6,7&cd=&sa;=&sdn=&lang=en-US).

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd

In [2]:
# Store the directory above the current directory
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Set the directory for where raw data files will be stored
raw_path = path + '/data/raw/'

# Set the directory for the final data files will be stored
processed_path = path + '/data/processed/'

## Travel Times - Part I

32 csv files were pulled from Uber Movement. The following code combines them and removes irrelevant columns.

In [3]:
# Combine all Uber Movement csv files into one dataframe.
df_uber = pd.DataFrame()

# Loop through the csv files using an int as the "year" portion of the file name.
for year in range(2016, 2020):
    
    # Loop through the csv files using an int as the "quarter" portion of the file name.
    for quarter in range(1, 5):
        
        # Store the data from the csv files for 'weekdays' travel times.
        file = 'washington_DC-censustracts-' + str(year) + '-' + str(quarter) + '-OnlyWeekdays-MonthlyAggregate.csv'
        df_weekdays = pd.read_csv(raw_path + file)
        df_weekdays['year'] = year
        df_weekdays['time_of_week'] = 'weekdays'
        
        # Store the data from the csv files for 'weekends' travel times.
        file = 'washington_DC-censustracts-' + str(year) + '-' + str(quarter) + '-OnlyWeekends-MonthlyAggregate.csv'
        df_weekends = pd.read_csv(raw_path + file)
        df_weekends['year'] = year
        df_weekends['time_of_week'] = 'weekends'
        
        # Combine the datasets.
        df_uber = pd.concat([df_uber, df_weekdays, df_weekends])

# Create a 'date' column to allow for time series modeling.
df_uber['day'] = 1
df_uber['date'] = pd.to_datetime(df_uber[['year','month','day']])

# Create a 'trip' column to allow for a unique model to be built depending on the start and end location.
df_uber['trip'] = df_uber.sourceid.astype(str) + '-' + df_uber.dstid.astype(str)

# Drop irrelevant colmuns from the dataset.
df_uber.drop(columns = ['standard_deviation_travel_time', 'geometric_mean_travel_time', 'geometric_standard_deviation_travel_time', 'year', 'month', 'day'], inplace = True)

## Census Tract Information

Uber uses standard census tract geometry for Washington DC however a different indicator was used to identify the tracts.  

A map of census tracts in Washington DC was used to create a reference. Those maps can be found here:
- North DC Census Tracts - https://www2.census.gov/geo/maps/dc10map/tract/st11_dc/c11001_district_of_columbia/DC10CT_C11001_001.pdf
- South DC Census Tracts - https://www2.census.gov/geo/maps/dc10map/tract/st11_dc/c11001_district_of_columbia/DC10CT_C11001_002.pdf

These were compared to the washington_DC_censustracts.json file pulled from Uber Movement found in the "data/raw" directory.

In [4]:
# Creating a dataframe with unique tracts.
df_tracts = pd.DataFrame()
df_tracts['uber_tract'] = df_uber.sourceid.unique()
df_tracts = df_tracts.uber_tract.sort_values().reset_index()
df_tracts.drop(columns = ['index'], inplace = True)
df_tracts['census_tract'] = 0

# Creating a dictionary where the keys are Uber ID's and the values are Washington DC census tract ID's
tract_dict = {1:73.01,2:16,3:9.02,4:15,5:8.02,6:13.01,7:62.02,8:47.02,9:26,10:1,11:18.03,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0,20:0,
         21:0,22:0,23:0,24:0,25:0,26:0,27:0,28:0,29:0,30:0,31:81,32:36,33:99.07,34:0,35:33.01,36:79.01,37:0,38:0,39:0,40:0,41:79.03,
         42:95.04,43:25.02,44:72,45:47.01,46:77.09,47:33.02,48:90,49:93.02,50:14.01,51:18.04,52:19.02,53:98.02,54:96.04,55:48.01,
         56:105,57:92.04,58:30,59:8.01,60:77.08,61:52.01,62:75.04,63:0,64:39,65:43,66:101,67:49.01,68:0,69:0,70:0,71:0,72:0,73:0,
         74:0,75:0,76:0,77:0,78:0,79:0,80:0,81:0,82:0,83:0,84:0,85:0,86:0,87:0,88:0,89:0,90:0,91:0,92:0,93:0,94:0,95:0,96:0,97:0,
         98:0,99:0,100:0,101:0,102:0,103:0,104:0,105:102,106:0,107:0,108:99.03,109:0,110:0,111:109,112:104,113:99.04,114:99.05,
         115:99.06,116:21.02,117:22.01,118:22.02,119:67,120:23.01,121:64,122:65,123:66,124:110,125:68.02,126:74.08,127:80.01,
         128:74.09,129:78.09,130:94,131:95.01,132:95.03,133:95.05,134:25.01,135:23.02,136:24,137:42.01,138:70,139:71,140:69,141:95.07,
         142:95.08,143:95.09,144:42.02,145:97,146:98.01,147:27.01,148:28.01,149:28.02,150:50.02,151:82,152:83.02,153:84.1,154:106,
         155:87.01,156:80.02,157:87.02,158:77.03,159:77.07,160:78.03,161:78.04,162:88.03,163:88.04,164:89.03,165:89.04,166:88.02,
         167:98.1,168:108,169:111,170:91.02,171:98.11,172:78.07,173:10.01,174:10.02,175:92.03,176:78.06,177:78.08,178:92.01,179:93.01,
         180:11,181:17.02,182:12,183:98.04,184:103,185:19.01,186:58,187:50.01,188:98.07,189:98.03,190:49.02,191:20.02,192:21.01,193:29,
         194:31,195:32,196:34,197:35,198:37,199:38,200:40.01,201:40.02,202:74.07,203:107,204:2.01,205:3,206:4,207:0,208:5.02,209:44,
         210:46,211:48.02,212:74.01,213:74.03,214:6,215:7.01,216:7.02,217:74.04,218:27.02,219:83.01,220:73.04,221:76.01,222:56,223:59,
         224:75.02,225:75.03,226:13.02,227:74.06,228:55,229:96.03,230:5.01,231:0,232:9.01,233:84.02,234:0,235:96.02,236:0,237:20.01,
         238:76.03,239:14.02,240:53.01,241:99.02,242:0,243:96.01,244:0,245:2.02,246:68.04,247:0,248:41,249:99.01,250:0,251:76.04,
         252:76.05,253:0,254:0,255:0,256:0,257:0,258:0,259:0,260:0,261:0,262:0,263:0,264:0,265:0,266:0,267:0,268:0,269:0,270:0,271:0,
         272:0,273:0,274:0,275:0,276:0,277:0,278:0,279:0,280:0,281:0,282:0,283:0,284:0,285:0,286:0,287:0,288:0,289:0,290:0,291:0,292:0,
         293:0,294:0,295:0,296:0,297:0,298:0,299:0,300:0,301:0,302:0,303:0,304:0,305:0,306:0,307:0,308:0,309:0,310:0,311:0,312:0,313:0,
         314:0,315:0,316:0,317:0,318:0,319:0,320:0,321:0,322:0,323:0,324:0,325:0,326:0,327:0,328:0,329:0,330:0,331:0,332:0,333:0,334:0,
         335:0,336:0,337:0,338:0,339:0,340:0,341:0,342:0,343:0,344:0,345:0,346:0,347:0,348:0,349:0,350:0,351:0,352:0,353:0,354:0,355:0,
         356:0,357:0,358:0,359:0,360:0,361:0,362:0,363:0,364:0,365:0,366:0,367:0,368:0,369:0,370:0,371:0,372:0,373:0,374:0,375:0,376:0,
         377:0,378:0,379:0,380:0,381:0,382:0,383:0,384:0,385:0,386:0,387:0,388:0,389:0,390:0,391:0,392:0,393:0,394:0,395:0,396:0,397:0,
         398:0,399:0,400:0,401:0,402:0,403:0,404:0,405:0,406:0,407:0,408:0,409:0,410:0,411:0,412:0,413:0,414:0,415:0,416:0,417:0,418:0,
         419:0,420:0,421:0,422:0,423:0,424:0,425:0,426:0,427:0,428:0,429:0,430:0,431:0,432:0,433:0,434:0,435:0,436:0,437:0,438:0,439:0,
         440:0,441:0,442:0,443:0,444:0,445:0,446:0,447:0,448:0,449:0,450:0,451:0,452:0,453:0,454:0,455:0,456:0,457:0,458:0,459:0,460:0,
         461:0,462:0,463:0,464:0,465:0,466:0,467:0,468:0,469:0,470:0,471:0,472:0,473:0,474:0,475:0,476:0,477:0,478:0,479:0,480:0,481:0,
         482:0,483:0,484:0,485:0,486:0,487:0,488:0,489:0,490:0,491:0,492:0,493:0,494:0,495:0,496:0,497:0,498:0,499:0,500:0,501:0,502:0,
         503:0,504:0,505:0,506:0,507:0,508:0,509:0,510:0,511:0,512:0,513:0,514:0,515:0,516:0,517:0,518:0,519:0,520:0,521:0,522:0,523:0,
         524:0,525:0,526:0,527:0,528:0,529:0,530:0,531:0,532:0,533:0,534:0,535:0,536:0,537:0,538:0,539:0,540:0,541:0,542:0,543:0,544:0,
         545:0,546:0,547:0,548:0,549:0,550:0,551:0,552:0,553:0,554:0,555:0,556:0,557:0,558:0}


for i in range(0, len(tract_dict)):
    df_tracts.census_tract[i] = '{:.2f}'.format(tract_dict.get(df_tracts.uber_tract[i]))

# Remove the Uber IDs from the list that don't have a Census Tract in DC
# Originally there were 558 Uber ID's included in the dataset for DC and the surrounding area
df_tracts = df_tracts[df_tracts['census_tract'] != '0.00']
df_tracts.census_tract = df_tracts.census_tract.astype(float)
df_tracts.to_csv(processed_path + 'census_tracts.csv', index=False)

## Travel Times - Part II

The Uber Movement dataset has trip information for tracts surrounding the DC area. These fall outside the scope of this project and were removed.

In [5]:
# Remove rows that have tracts that aren't located in DC and reset the index.
df_uber = df_uber[df_uber['sourceid'].isin(df_tracts['uber_tract'])]
df_uber = df_uber[df_uber['dstid'].isin(df_tracts['uber_tract'])]
df_uber.reset_index(drop = True, inplace = True)

## Save the dataset

In [6]:
df_uber.to_csv(processed_path + 'uber.csv', index=False)